<a href="https://colab.research.google.com/github/Zabiullahkhan/Data_Science/blob/main/Roberta_large_for_76_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets
!pip install transformers
# !pip install transformers[torch]
# !pip install accelerate -U
import transformers

print(transformers.__version__)

4.33.2


In [2]:
import os
import torch
import pandas as pd
import transformers
import tensorflow as tf
from sklearn.model_selection import train_test_split
from transformers import TextClassificationPipeline
from transformers import TFTrainingArguments, TFTrainer
from transformers import DistilBertTokenizer, DistilBertModel, TFRobertaForSequenceClassification,BertForSequenceClassification, BertTokenizerFast
from torch.utils.data import Dataset
from tqdm import tqdm

In [3]:
from transformers import pipeline

In [4]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cpu'

In [5]:
# !wget https://raw.githubusercontent.com/PolyAI-LDN/task-specific-datasets/master/banking_data/train.csv
# !wget https://raw.githubusercontent.com/PolyAI-LDN/task-specific-datasets/master/banking_data/test.csv
# !wget https://github.com/PolyAI-LDN/task-specific-datasets/blob/master/banking_data/categories.json

In [6]:
train_data = pd.read_csv('../content/train.csv')
test_data = pd.read_csv('../content/test.csv')
df = pd.concat([train_data, test_data], axis=0)
df = df.reset_index(drop=True)
df['label'] = df['category'].astype('category').cat.codes
id2label = {label:category for category, label in sorted(df[['category', 'label']].drop_duplicates().values, key = lambda x:x[1])}
label2id = {category:label for category, label in sorted(df[['category', 'label']].drop_duplicates().values, key = lambda x:x[1])}

NUM_LABELS = len(df['label'].drop_duplicates())
NUM_LABELS

77

In [7]:
# df.category.value_counts().plot(kind ='pie', figsize = (10,10))

In [8]:
df.head(2)

,text,category,label
0,I am still waiting on my card?,card_arrival,12
1,What can I do if my card still hasn't arrived ...,card_arrival,12


In [9]:
data_texts = df['text'].to_list()
data_labels = df['label'].to_list()


train_texts, val_texts, train_labels, val_labels = train_test_split(data_texts, data_labels, test_size= 0.2)
train_texts, test_texts, train_labels, test_labels = train_test_split(train_texts, train_labels, test_size = 0.1)

In [10]:
from transformers import RobertaTokenizer, TFRobertaModel
tokenizer = RobertaTokenizer.from_pretrained('roberta-large', max_length = 512)
model = TFRobertaModel.from_pretrained('roberta-large', num_labels = NUM_LABELS)
train_encodings = tokenizer(train_texts, truncation = True, padding = True)
val_encodings = tokenizer(val_texts, truncation = True, padding = True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'roberta.embeddings.position_ids', 'lm_head.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaModel were not initialized from the PyTorch model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infe

In [11]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
))

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
))

In [12]:
# tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased", max_length = 512)
# model = DistilBertModel.from_pretrained("distilbert-base-uncased", num_labels = NUM_LABELS)


In [13]:
# model.to(device)

In [14]:
training_args = TFTrainingArguments(
    output_dir = './results',
    num_train_epochs = 1,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 64,
    warmup_steps = 500,
    weight_decay = 1e-5,
    logging_dir = './logs',
    eval_steps = 100

)


In [15]:
with training_args.strategy.scope():
  trainer_model = TFRobertaForSequenceClassification.from_pretrained('roberta-large',num_labels = NUM_LABELS)#, num_labels = NUM_LABELS
trainer = TFTrainer(
    model = trainer_model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti

In [ ]:
trainer.train()
trainer.evaluate()

In [ ]:

save_directory = "/save_models"

model.save_pretrained(save_directory)

tokenizer.save_pretrained(save_directory)